This notebook will do text processing for StumbleUpon data.
- First it take out JSON parameters in ```boilerplate``` into columns
- Use NTLK to tokenize `title` and ```body``` and then remove stopwords
- Next step is calculating sentiment for title & body -> however this doesn't affect prediction, so it's not necessary
- Lastly, create n-grams of the text using NTLK

## first, separate JSON components

In [5]:
import pandas as pd
from textblob import TextBlob
import json
df = pd.read_csv("Stumble_Upon/test_2grams.tsv", sep="\t")
df.head()

,Unnamed: 0,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,...,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,title_3grams,body_3grams,url_3grams
0,0,http://www.lynnskitchenadventures.com/2009/04/...,5865,"{""title"":""Homemade Enchilada Sauce Lynn s Kitc...",recreation,0.443906,2.558140,0.389706,0.257353,0.044118,...,15,0,5643,136,3,0.242647,0.080597,"['homemade enchilada', 'enchilada sauce', 'sau...","['usually buy', 'buy enchilada', 'enchilada sa...","['lynnskitchenadventures 2009', '2009 04', '04..."
1,1,http://lolpics.se/18552-stun-grenade-ar,782,"{""title"":""lolpics Stun grenade ar "",""body"":"" f...",culture_politics,0.135844,3.771429,0.461538,0.205128,0.051282,...,62,0,382,39,2,0.128205,0.176471,"['lolpics stun', 'stun grenade', 'grenade ar']","['funny pictures', 'pictures lolpics.se', 'lol...","['lolpics se', 'se 18552', '18552 stun', 'stun..."
2,2,http://www.xcelerationfitness.com/treadmills.html,6962,"{""title"":""Treadmills "",""body"":"" treadmills, st...",?,?,2.269565,0.495726,0.384615,0.170940,...,42,0,2420,117,1,0.581197,0.125000,[],"['treadmills stair', 'stair climbers', 'climbe...",['xcelerationfitness treadmills']
3,3,http://www.bloomberg.com/news/2012-02-06/syria...,7640,"{""title"":""Father s Tactics Used by Assad to Cr...",culture_politics,0.90259,2.523490,0.705502,0.346278,0.122977,...,41,0,5559,309,10,0.038835,0.063126,"['father tactics', 'tactics used', 'used assad...","['enlarge image', 'image syrian', 'syrian pres...","['bloomberg news', 'news 2012', '2012 02', '02..."
4,4,http://www.wired.com/gadgetlab/2011/12/stem-tu...,3589,"{""title"":""Stem Turns Lemons and Limes Into Jui...",science_technology,0.486363,1.848000,0.470968,0.161290,0.032258,...,34,0,2209,155,10,0.096774,0.065341,"['stem turns', 'turns lemons', 'lemons limes',...","['quirky stem', 'stem turns', 'turns citrus', ...","['wired gadgetlab', 'gadgetlab 2011', '2011 12..."


In [ ]:
df.head().to_csv('test_5rows.tsv', sep='\t')

In [61]:
df['boilerplate'][1]

'{"title":"The Fully Electronic Futuristic Starting Gun That Eliminates Advantages in Races the fully electronic, futuristic starting gun that eliminates advantages in races the fully electronic, futuristic starting gun that eliminates advantages in races","body":"And that can be carried on a plane without the hassle too The Omega E Gun Starting Pistol Omega It s easy to take for granted just how insanely close some Olympic races are and how much the minutiae of it all can matter The perfect example is the traditional starting gun Seems easy You pull a trigger and the race starts Boom What people don t consider When a conventional gun goes off the sound travels to the ears of the closest runner a fraction of a second sooner than the others That s just enough to matter and why the latest starting pistol has traded in the mechanical boom for orchestrated electronic noise Omega has been the watch company tasked as the official timekeeper of the Olympic Games since 1932 At the 2010 Vancouv

Load json object as dictionary, and append to list

In [224]:
len(boiler)

7395

In [219]:
boiler = []
for text in df['boilerplate']:
    jsn = json.loads(text)
    boiler.append(jsn)
boiler[0]

{'body': 'A sign stands outside the International Business Machines Corp IBM Almaden Research Center campus in San Jose California Photographer Tony Avelar Bloomberg Buildings stand at the International Business Machines Corp IBM Almaden Research Center campus in the Santa Teresa Hills of San Jose California Photographer Tony Avelar Bloomberg By 2015 your mobile phone will project a 3 D image of anyone who calls and your laptop will be powered by kinetic energy At least that s what International Business Machines Corp sees in its crystal ball The predictions are part of an annual tradition for the Armonk New York based company which surveys its 3 000 researchers to find five ideas expected to take root in the next five years IBM the world s largest provider of computer services looks to Silicon Valley for input gleaning many ideas from its Almaden research center in San Jose California Holographic conversations projected from mobile phones lead this year s list The predictions also inc

transform body, title, url in boilerplate as column

In [174]:
a = 'its a'
b = 'sun'
a+' ' +b

'its a sun'

In [178]:
type(boiler[1].get('title'))

str

In [184]:
boiler[1].get('body')

'And that can be carried on a plane without the hassle too The Omega E Gun Starting Pistol Omega It s easy to take for granted just how insanely close some Olympic races are and how much the minutiae of it all can matter The perfect example is the traditional starting gun Seems easy You pull a trigger and the race starts Boom What people don t consider When a conventional gun goes off the sound travels to the ears of the closest runner a fraction of a second sooner than the others That s just enough to matter and why the latest starting pistol has traded in the mechanical boom for orchestrated electronic noise Omega has been the watch company tasked as the official timekeeper of the Olympic Games since 1932 At the 2010 Vancouver games they debuted their new starting gun which is a far cry from the iconic revolvers associated with early games it s clearly electronic but still more than a button that s pressed to get the show rolling About as far away as you can get probably while still 

In [249]:
b=df['boiler_text']


In [250]:
boiler_text=[]
for text in b:
    if text is not None:
        words = ' '.join(text)
        boiler_text.append(words)
    else:
        boiler_text.append(None)
        
se_bo=pd.Series(boiler_text)
len(boiler_text)
len(se_bo)
df['boiler_text']=se_bo.values
df

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,lengthyLinkDomain,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label,boiler_text
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,...,1,24,0,5424,170,8,0.152941,0.079130,0,sign stands outside international business mac...
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,...,1,40,0,4973,187,9,0.181818,0.125448,1,carried plane without hassle omega e gun start...
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,...,1,55,0,2240,258,11,0.166667,0.057613,1,apples popular source antioxidants diet one ap...
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,...,0,24,0,2737,120,5,0.041667,0.100858,1,period life lot problems sleep took long fall ...
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,...,1,14,0,12032,162,10,0.098765,0.082569,0,jersey sales curious business whether buying s...
5,http://www.conveniencemedical.com/genital-herp...,7018,"{""url"":""conveniencemedical genital herpes home...",?,?,119.000000,0.745455,0.581818,0.290909,0.018182,...,0,12,?,4368,55,3,0.054545,0.087356,0,genital herpes caused herpes simplex virus hsv...
6,http://gofashionlane.blogspot.tw/2012/06/ameri...,8685,"{""title"":""fashion lane American Wild Child "",""...",arts_entertainment,0.22111,0.773810,0.215054,0.053763,0.043011,0.043011,...,0,21,0,1287,93,3,0.548387,0.064327,1,favorite summer holiday around corner boy heav...
7,http://www.insidershealth.com/article/racing_f...,3402,"{""url"":""insidershealth article racing for reco...",?,?,1.883333,0.719697,0.265152,0.113636,0.015152,...,0,5,?,27656,132,4,0.068182,0.148551,0,racing recovery growing idea drug addiction al...
8,http://www.valetmag.com/the-handbook/features/...,477,"{""title"":""Valet The Handbook 31 Days 31 days"",...",?,?,0.471503,0.190722,0.036082,0.000000,0.000000,...,0,17,0,2471,194,7,0.644330,0.125000,1,resolutions suckers instead swearing sugar two...
9,http://www.howsweeteats.com/2010/03/24/cookies...,6731,"{""url"":""howsweeteats 2010 03 24 cookies and cr...",?,?,2.410112,0.469325,0.101227,0.018405,0.003067,...,1,14,?,11459,326,4,0.236196,0.094412,1,brownies seems get one full week without tryin...


In [239]:
ignorechars = [',', '.','-','--', '&', ';', ':', '?','#','>','<','=','[',']',')','(', '|','com','html', '!']
filtered_text=[]
for text in boilerplate:
    if text is not None:
        tokens = get_tokens(text)
        filtered = [w for w in tokens if not w in stopwords.words('english')]
        filtered = [w for w in filtered if not w in ignorechars]
        filtered_text.append(filtered)
    else:
        filtered_text.append(None)
    
se_boiler=pd.Series(filtered_text)
df['boiler_text']=se_boiler.values
df

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,lengthyLinkDomain,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label,boiler_text
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,...,1,24,0,5424,170,8,0.152941,0.079130,0,"[sign, stands, outside, international, busines..."
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,...,1,40,0,4973,187,9,0.181818,0.125448,1,"[carried, plane, without, hassle, omega, e, gu..."
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,...,1,55,0,2240,258,11,0.166667,0.057613,1,"[apples, popular, source, antioxidants, diet, ..."
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,...,0,24,0,2737,120,5,0.041667,0.100858,1,"[period, life, lot, problems, sleep, took, lon..."
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,...,1,14,0,12032,162,10,0.098765,0.082569,0,"[jersey, sales, curious, business, whether, bu..."
5,http://www.conveniencemedical.com/genital-herp...,7018,"{""url"":""conveniencemedical genital herpes home...",?,?,119.000000,0.745455,0.581818,0.290909,0.018182,...,0,12,?,4368,55,3,0.054545,0.087356,0,"[genital, herpes, caused, herpes, simplex, vir..."
6,http://gofashionlane.blogspot.tw/2012/06/ameri...,8685,"{""title"":""fashion lane American Wild Child "",""...",arts_entertainment,0.22111,0.773810,0.215054,0.053763,0.043011,0.043011,...,0,21,0,1287,93,3,0.548387,0.064327,1,"[favorite, summer, holiday, around, corner, bo..."
7,http://www.insidershealth.com/article/racing_f...,3402,"{""url"":""insidershealth article racing for reco...",?,?,1.883333,0.719697,0.265152,0.113636,0.015152,...,0,5,?,27656,132,4,0.068182,0.148551,0,"[racing, recovery, growing, idea, drug, addict..."
8,http://www.valetmag.com/the-handbook/features/...,477,"{""title"":""Valet The Handbook 31 Days 31 days"",...",?,?,0.471503,0.190722,0.036082,0.000000,0.000000,...,0,17,0,2471,194,7,0.644330,0.125000,1,"[resolutions, suckers, instead, swearing, suga..."
9,http://www.howsweeteats.com/2010/03/24/cookies...,6731,"{""url"":""howsweeteats 2010 03 24 cookies and cr...",?,?,2.410112,0.469325,0.101227,0.018405,0.003067,...,1,14,?,11459,326,4,0.236196,0.094412,1,"[brownies, seems, get, one, full, week, withou..."


In [238]:
# add all json params to one list
boilerplate=[]
for item in boiler:
    if (item.get('body') and item.get('title')) is not None:
        boilerplate.append((item.get('body')+ ' '+item.get('title')))
    elif (item.get('title') is None and item.get('body') is not None):
        boilerplate.append(item.get('body'))
    elif (item.get('title') is not None and item.get('body') is None):
        boilerplate.append(item.get('title'))
    else:
        boilerplate.append(None)
se_boiler=pd.Series(boilerplate)
df['boiler_text']=se_boiler.values
df

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,lengthyLinkDomain,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label,boiler_text
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,...,1,24,0,5424,170,8,0.152941,0.079130,0,A sign stands outside the International Busine...
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,...,1,40,0,4973,187,9,0.181818,0.125448,1,And that can be carried on a plane without the...
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,...,1,55,0,2240,258,11,0.166667,0.057613,1,Apples The most popular source of antioxidants...
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,...,0,24,0,2737,120,5,0.041667,0.100858,1,There was a period in my life when I had a lot...
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,...,1,14,0,12032,162,10,0.098765,0.082569,0,Jersey sales is a curious business Whether you...
5,http://www.conveniencemedical.com/genital-herp...,7018,"{""url"":""conveniencemedical genital herpes home...",?,?,119.000000,0.745455,0.581818,0.290909,0.018182,...,0,12,?,4368,55,3,0.054545,0.087356,0,Genital herpes is caused by herpes simplex vir...
6,http://gofashionlane.blogspot.tw/2012/06/ameri...,8685,"{""title"":""fashion lane American Wild Child "",""...",arts_entertainment,0.22111,0.773810,0.215054,0.053763,0.043011,0.043011,...,0,21,0,1287,93,3,0.548387,0.064327,1,Our favorite summer holiday is just around the...
7,http://www.insidershealth.com/article/racing_f...,3402,"{""url"":""insidershealth article racing for reco...",?,?,1.883333,0.719697,0.265152,0.113636,0.015152,...,0,5,?,27656,132,4,0.068182,0.148551,0,Racing For Recovery is the growing idea that d...
8,http://www.valetmag.com/the-handbook/features/...,477,"{""title"":""Valet The Handbook 31 Days 31 days"",...",?,?,0.471503,0.190722,0.036082,0.000000,0.000000,...,0,17,0,2471,194,7,0.644330,0.125000,1,Resolutions are for suckers Instead of swearin...
9,http://www.howsweeteats.com/2010/03/24/cookies...,6731,"{""url"":""howsweeteats 2010 03 24 cookies and cr...",?,?,2.410112,0.469325,0.101227,0.018405,0.003067,...,1,14,?,11459,326,4,0.236196,0.094412,1,More brownies It seems that I can t get throug...


In [69]:
body=[]
title=[]
url=[]
for item in boiler:
    body.append(item.get('body'))
    title.append(item.get('title'))
    url.append(item.get('url'))
title

['IBM Sees Holographic Calls Air Breathing Batteries ibm sees holographic calls, air-breathing batteries',
 'The Fully Electronic Futuristic Starting Gun That Eliminates Advantages in Races the fully electronic, futuristic starting gun that eliminates advantages in races the fully electronic, futuristic starting gun that eliminates advantages in races',
 "Fruits that Fight the Flu fruits that fight the flu | cold & flu | men's health",
 '10 Foolproof Tips for Better Sleep ',
 "The 50 Coolest Jerseys You Didn t Know Existed coolest jerseys you haven't seen",
 'Genital Herpes Treatment ',
 'fashion lane American Wild Child ',
 'Racing For Recovery by Dean Johnson racing for recovery by dean johnson - - insidershealth.com',
 'Valet The Handbook 31 Days 31 days',
 'Cookies and Cream Brownies How Sweet It Is ',
 'Business Financial News Breaking US International News ',
 'A Tip of the Cap to The Greatest Iron Man of Them All ',
 '9 Foods That Trash Your Teeth ',
 ' ',
 'French Onion Steaks 

In [51]:
title_sa = pd.Series(body_polarity)
body_sa = pd.Series(body_polarity)
df['title_sentiment'] = title_sa.values
df['body_sentiment'] = body_sa.values
df

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label,boiler_body,boiler_title,boiler_url,title_sentiment,body_sentiment
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,...,170,8,0.152941,0.079130,0,A sign stands outside the International Busine...,IBM Sees Holographic Calls Air Breathing Batte...,bloomberg news 2010 12 23 ibm predicts hologra...,0.096909,0.096909
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,...,187,9,0.181818,0.125448,1,And that can be carried on a plane without the...,The Fully Electronic Futuristic Starting Gun T...,popsci technology article 2012 07 electronic f...,0.168040,0.168040
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,...,258,11,0.166667,0.057613,1,Apples The most popular source of antioxidants...,Fruits that Fight the Flu fruits that fight th...,menshealth health flu fighting fruits cm mmc F...,0.029092,0.029092
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,...,120,5,0.041667,0.100858,1,There was a period in my life when I had a lot...,10 Foolproof Tips for Better Sleep,dumblittleman 2007 12 10 foolproof tips for be...,0.060979,0.060979
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,...,162,10,0.098765,0.082569,0,Jersey sales is a curious business Whether you...,The 50 Coolest Jerseys You Didn t Know Existed...,bleacherreport articles 1205138 the 50 coolest...,0.162464,0.162464
5,http://www.conveniencemedical.com/genital-herp...,7018,"{""url"":""conveniencemedical genital herpes home...",?,?,119.000000,0.745455,0.581818,0.290909,0.018182,...,55,3,0.054545,0.087356,0,Genital herpes is caused by herpes simplex vir...,Genital Herpes Treatment,conveniencemedical genital herpes home php,0.061286,0.061286
6,http://gofashionlane.blogspot.tw/2012/06/ameri...,8685,"{""title"":""fashion lane American Wild Child "",""...",arts_entertainment,0.22111,0.773810,0.215054,0.053763,0.043011,0.043011,...,93,3,0.548387,0.064327,1,Our favorite summer holiday is just around the...,fashion lane American Wild Child,gofashionlane blogspot tw 2012 06 american wil...,0.265909,0.265909
7,http://www.insidershealth.com/article/racing_f...,3402,"{""url"":""insidershealth article racing for reco...",?,?,1.883333,0.719697,0.265152,0.113636,0.015152,...,132,4,0.068182,0.148551,0,Racing For Recovery is the growing idea that d...,Racing For Recovery by Dean Johnson racing for...,insidershealth article racing for recovery 3471,0.239091,0.239091
8,http://www.valetmag.com/the-handbook/features/...,477,"{""title"":""Valet The Handbook 31 Days 31 days"",...",?,?,0.471503,0.190722,0.036082,0.000000,0.000000,...,194,7,0.644330,0.125000,1,Resolutions are for suckers Instead of swearin...,Valet The Handbook 31 Days 31 days,valetmag the handbook features 31 days index p...,0.158475,0.158475
9,http://www.howsweeteats.com/2010/03/24/cookies...,6731,"{""url"":""howsweeteats 2010 03 24 cookies and cr...",?,?,2.410112,0.469325,0.101227,0.018405,0.003067,...,326,4,0.236196,0.094412,1,More brownies It seems that I can t get throug...,Cookies and Cream Brownies How Sweet It Is,howsweeteats 2010 03 24 cookies and cream brow...,0.213948,0.213948


In [52]:
df.to_csv("train-stumble_upon_set2.tsv", sep="\t")

In [53]:
new_df=pd.read_csv("train-stumble_upon_set2.tsv", sep="\t")
new_df

,Unnamed: 0,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,...,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label,boiler_body,boiler_title,boiler_url,title_sentiment,body_sentiment
0,0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,...,170,8,0.152941,0.079130,0,A sign stands outside the International Busine...,IBM Sees Holographic Calls Air Breathing Batte...,bloomberg news 2010 12 23 ibm predicts hologra...,0.096909,0.096909
1,1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,...,187,9,0.181818,0.125448,1,And that can be carried on a plane without the...,The Fully Electronic Futuristic Starting Gun T...,popsci technology article 2012 07 electronic f...,0.168040,0.168040
2,2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,...,258,11,0.166667,0.057613,1,Apples The most popular source of antioxidants...,Fruits that Fight the Flu fruits that fight th...,menshealth health flu fighting fruits cm mmc F...,0.029092,0.029092
3,3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,...,120,5,0.041667,0.100858,1,There was a period in my life when I had a lot...,10 Foolproof Tips for Better Sleep,dumblittleman 2007 12 10 foolproof tips for be...,0.060979,0.060979
4,4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,...,162,10,0.098765,0.082569,0,Jersey sales is a curious business Whether you...,The 50 Coolest Jerseys You Didn t Know Existed...,bleacherreport articles 1205138 the 50 coolest...,0.162464,0.162464
5,5,http://www.conveniencemedical.com/genital-herp...,7018,"{""url"":""conveniencemedical genital herpes home...",?,?,119.000000,0.745455,0.581818,0.290909,...,55,3,0.054545,0.087356,0,Genital herpes is caused by herpes simplex vir...,Genital Herpes Treatment,conveniencemedical genital herpes home php,0.061286,0.061286
6,6,http://gofashionlane.blogspot.tw/2012/06/ameri...,8685,"{""title"":""fashion lane American Wild Child "",""...",arts_entertainment,0.22111,0.773810,0.215054,0.053763,0.043011,...,93,3,0.548387,0.064327,1,Our favorite summer holiday is just around the...,fashion lane American Wild Child,gofashionlane blogspot tw 2012 06 american wil...,0.265909,0.265909
7,7,http://www.insidershealth.com/article/racing_f...,3402,"{""url"":""insidershealth article racing for reco...",?,?,1.883333,0.719697,0.265152,0.113636,...,132,4,0.068182,0.148551,0,Racing For Recovery is the growing idea that d...,Racing For Recovery by Dean Johnson racing for...,insidershealth article racing for recovery 3471,0.239091,0.239091
8,8,http://www.valetmag.com/the-handbook/features/...,477,"{""title"":""Valet The Handbook 31 Days 31 days"",...",?,?,0.471503,0.190722,0.036082,0.000000,...,194,7,0.644330,0.125000,1,Resolutions are for suckers Instead of swearin...,Valet The Handbook 31 Days 31 days,valetmag the handbook features 31 days index p...,0.158475,0.158475
9,9,http://www.howsweeteats.com/2010/03/24/cookies...,6731,"{""url"":""howsweeteats 2010 03 24 cookies and cr...",?,?,2.410112,0.469325,0.101227,0.018405,...,326,4,0.236196,0.094412,1,More brownies It seems that I can t get throug...,Cookies and Cream Brownies How Sweet It Is,howsweeteats 2010 03 24 cookies and cream brow...,0.213948,0.213948


## USE NTLK TO REMOVE STOPWORDS AND CHARS

In [70]:
import nltk
import string
nltk.download('punkt')
from nltk.corpus import stopwords
from collections import Counter

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dindatisicalista/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [71]:
def get_tokens(rev):
    lowers = str(rev).lower() 
    #remove the punctuation using the character deletion step of translate
    no_punctuation = lowers.translate(string.punctuation)
    tokens = nltk.word_tokenize(no_punctuation)
    return tokens

In [38]:
title[17] is not None

False

In [97]:
ignorechars = [',', '.','-','--', '&', ';', ':', '?','#','>','<','=','[',']',')','(', '|','com','html', '!']
filtered_title=[]
for text in title:
    if text is not None:
        tokens = get_tokens(text)
        filtered = [w for w in tokens if not w in stopwords.words('english')]
        filtered = [w for w in filtered if not w in ignorechars]
        filtered_title.append(filtered)
    else:
        filtered_title.append(None)
    
filtered_title

[['ibm',
  'sees',
  'holographic',
  'calls',
  'air',
  'breathing',
  'batteries',
  'ibm',
  'sees',
  'holographic',
  'calls',
  'air-breathing',
  'batteries'],
 ['fully',
  'electronic',
  'futuristic',
  'starting',
  'gun',
  'eliminates',
  'advantages',
  'races',
  'fully',
  'electronic',
  'futuristic',
  'starting',
  'gun',
  'eliminates',
  'advantages',
  'races',
  'fully',
  'electronic',
  'futuristic',
  'starting',
  'gun',
  'eliminates',
  'advantages',
  'races'],
 ['fruits',
  'fight',
  'flu',
  'fruits',
  'fight',
  'flu',
  'cold',
  'flu',
  'men',
  "'s",
  'health'],
 ['10', 'foolproof', 'tips', 'better', 'sleep'],
 ['50',
  'coolest',
  'jerseys',
  'know',
  'existed',
  'coolest',
  'jerseys',
  "n't",
  'seen'],
 ['genital', 'herpes', 'treatment'],
 ['fashion', 'lane', 'american', 'wild', 'child'],
 ['racing',
  'recovery',
  'dean',
  'johnson',
  'racing',
  'recovery',
  'dean',
  'johnson',
  'insidershealth.com'],
 ['valet', 'handbook', '31',

In [98]:
ignorechars = [',', '.','-','--', '&', ';', ':', '?','#','>','<','=','[',']',')','(', '|','com','html','!']
filtered_url=[]
for text in url:
    if text is not None:
        tokens = get_tokens(text)
        filtered = [w for w in tokens if not w in stopwords.words('english')]
        filtered = [w for w in filtered if not w in ignorechars]
        filtered_url.append(filtered)
    else:
        filtered_url.append(None)
filtered_url[3]

['dumblittleman', '2007', '12', '10', 'foolproof', 'tips', 'better', 'sleep']

In [118]:
body[1]

'And that can be carried on a plane without the hassle too The Omega E Gun Starting Pistol Omega It s easy to take for granted just how insanely close some Olympic races are and how much the minutiae of it all can matter The perfect example is the traditional starting gun Seems easy You pull a trigger and the race starts Boom What people don t consider When a conventional gun goes off the sound travels to the ears of the closest runner a fraction of a second sooner than the others That s just enough to matter and why the latest starting pistol has traded in the mechanical boom for orchestrated electronic noise Omega has been the watch company tasked as the official timekeeper of the Olympic Games since 1932 At the 2010 Vancouver games they debuted their new starting gun which is a far cry from the iconic revolvers associated with early games it s clearly electronic but still more than a button that s pressed to get the show rolling About as far away as you can get probably while still 

In [119]:
ignorechars = [',', '.','-','--', '&', ';', ':', '?','#','>','<','=','[',']',')','(', '|','com','html']
filtered_body=[]
for text in body:
    if text is not None:
        tokens = get_tokens(text)
        filtered = [w for w in tokens if not w in ignorechars]
        filtered = [w for w in filtered if not w in stopwords.words('english')]
        filtered_body.append(filtered)
    else:
        filtered_body.append(None)
filtered_body[3]

['period',
 'life',
 'lot',
 'problems',
 'sleep',
 'took',
 'long',
 'fall',
 'asleep',
 'easily',
 'awaken',
 'simply',
 'getting',
 'enough',
 'rest',
 'night',
 'want',
 'take',
 'medication',
 'led',
 'learn',
 'several',
 'tips',
 'tricks',
 'really',
 'helped',
 'overcome',
 'insomnia',
 'tips',
 'try',
 'follow',
 'regularly',
 'worry',
 'getting',
 'enough',
 'sleep',
 'try',
 'worry',
 'much',
 'sleep',
 'worrying',
 'start',
 'cycle',
 'negative',
 'thoughts',
 'contribute',
 'condition',
 'known',
 'learned',
 'insomnia',
 'learned',
 'insomnia',
 'occurs',
 'worry',
 'much',
 'whether',
 'able',
 'get',
 'adequate',
 'sleep',
 'bedtime',
 'rituals',
 'behavior',
 'actually',
 'trigger',
 'insomnia',
 'force',
 'sleep',
 'attempt',
 'trying',
 'actually',
 'awakes',
 'making',
 'difficult',
 'sleep',
 'go',
 'bed',
 'feeling',
 'really',
 'tired',
 'sleepy',
 'look',
 'alarm',
 'clock',
 'night',
 'looking',
 'clock',
 'promotes',
 'increased',
 'anxiety',
 'obsession',
 't

In [100]:
boiler_url = []

for text in filtered_url:
    if text is not None:
        words = ' '.join(text)
        boiler_url.append(words)
    else:
        boiler_url.append(None)



In [101]:
boiler_title = []

for text in filtered_title:
    if text is not None:
        words = ' '.join(text)
        boiler_title.append(words)
    else:
        boiler_title.append(None)

print(boiler_title[1])


boiler_body = []
for text in filtered_body:
    if text is not None:
        words = ' '.join(text)
        boiler_body.append(words)
    else:
        boiler_body.append(None)
print(boiler_body[1])

fully electronic futuristic starting gun eliminates advantages races fully electronic futuristic starting gun eliminates advantages races fully electronic futuristic starting gun eliminates advantages races
and that can be carried on a plane without the hassle too the omega e gun starting pistol omega it s easy to take for granted just how insanely close some olympic races are and how much the minutiae of it all can matter the perfect example is the traditional starting gun seems easy you pull a trigger and the race starts boom what people don t consider when a conventional gun goes off the sound travels to the ears of the closest runner a fraction of a second sooner than the others that s just enough to matter and why the latest starting pistol has traded in the mechanical boom for orchestrated electronic noise omega has been the watch company tasked as the official timekeeper of the olympic games since 1932 at the 2010 vancouver games they debuted their new starting gun which is a fa

In [78]:
se_body = pd.Series(boiler_body)
se_title = pd.Series(boiler_title)
se_url = pd.Series(boiler_url)
df['boiler_body'] = se_body.values
df['boiler_title'] = se_title.values
df['boiler_url'] = se_url.values
df.to_csv("train-stumble_upon_boiler-filtered.tsv", sep="\t")
df

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label,boiler_body,boiler_title,boiler_url
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,...,0,5424,170,8,0.152941,0.079130,0,a sign stands outside the international busine...,ibm sees holographic calls air breathing batte...,bloomberg news 2010 12 23 ibm predicts hologra...
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,...,0,4973,187,9,0.181818,0.125448,1,and that can be carried on a plane without the...,the fully electronic futuristic starting gun t...,popsci technology article 2012 07 electronic f...
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,...,0,2240,258,11,0.166667,0.057613,1,apples the most popular source of antioxidants...,fruits that fight the flu fruits that fight th...,menshealth health flu fighting fruits cm mmc f...
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,...,0,2737,120,5,0.041667,0.100858,1,there was a period in my life when i had a lot...,10 foolproof tips for better sleep,dumblittleman 2007 12 10 foolproof tips for be...
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,...,0,12032,162,10,0.098765,0.082569,0,jersey sales is a curious business whether you...,the 50 coolest jerseys you didn t know existed...,bleacherreport articles 1205138 the 50 coolest...
5,http://www.conveniencemedical.com/genital-herp...,7018,"{""url"":""conveniencemedical genital herpes home...",?,?,119.000000,0.745455,0.581818,0.290909,0.018182,...,?,4368,55,3,0.054545,0.087356,0,genital herpes is caused by herpes simplex vir...,genital herpes treatment,conveniencemedical genital herpes home php
6,http://gofashionlane.blogspot.tw/2012/06/ameri...,8685,"{""title"":""fashion lane American Wild Child "",""...",arts_entertainment,0.22111,0.773810,0.215054,0.053763,0.043011,0.043011,...,0,1287,93,3,0.548387,0.064327,1,our favorite summer holiday is just around the...,fashion lane american wild child,gofashionlane blogspot tw 2012 06 american wil...
7,http://www.insidershealth.com/article/racing_f...,3402,"{""url"":""insidershealth article racing for reco...",?,?,1.883333,0.719697,0.265152,0.113636,0.015152,...,?,27656,132,4,0.068182,0.148551,0,racing for recovery is the growing idea that d...,racing for recovery by dean johnson racing for...,insidershealth article racing for recovery 3471
8,http://www.valetmag.com/the-handbook/features/...,477,"{""title"":""Valet The Handbook 31 Days 31 days"",...",?,?,0.471503,0.190722,0.036082,0.000000,0.000000,...,0,2471,194,7,0.644330,0.125000,1,resolutions are for suckers instead of swearin...,valet the handbook 31 days 31 days,valetmag the handbook features 31 days index p...
9,http://www.howsweeteats.com/2010/03/24/cookies...,6731,"{""url"":""howsweeteats 2010 03 24 cookies and cr...",?,?,2.410112,0.469325,0.101227,0.018405,0.003067,...,?,11459,326,4,0.236196,0.094412,1,more brownies it seems that i can t get throug...,cookies and cream brownies how sweet it is,howsweeteats 2010 03 24 cookies and cream brow...


In [ ]:
# display result
texts_ngrams.select('2-grams').show(truncate=False)
texts_ngrams.select('3-grams').show(truncate=False)
texts_ngrams.select('4-grams').show(truncate=False)

## n-grams

In [102]:
from nltk import ngrams
sentence = 'this is a foo bar sentences and i want to ngramize it'
n = 3
gramlist=[]
sixgrams = ngrams(sentence.split(), n)
for grams in sixgrams:
    gramlist.append(grams)
gramlist

[('this', 'is', 'a'),
 ('is', 'a', 'foo'),
 ('a', 'foo', 'bar'),
 ('foo', 'bar', 'sentences'),
 ('bar', 'sentences', 'and'),
 ('sentences', 'and', 'i'),
 ('and', 'i', 'want'),
 ('i', 'want', 'to'),
 ('want', 'to', 'ngramize'),
 ('to', 'ngramize', 'it')]

In [103]:

gramlist=[]
for text in boiler_title:
    if text is not None:
        three_grams=ngrams(text.split(), 3)
        grams= [' '.join(gram) for gram in three_grams]
        gramlist.append(grams)
    else:
        gramlist.append(None)
                
grams=[] # cleanup for next trial
title_3grams = pd.Series(gramlist)
len(gramlist)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: generator 'ngrams' raised StopIteration
  


7395

In [109]:
#url3-grams
gramlist=[]
for text in boiler_url:
    if text is not None:
        three_grams=ngrams(text.split(), 32)
        grams= [' '.join(gram) for gram in three_grams]
        gramlist.append(grams)
    else:
        gramlist.append(None)
                
grams=[] # cleanup for next trial
url_3grams = pd.Series(gramlist)
len(gramlist)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: generator 'ngrams' raised StopIteration
  


7395

In [87]:
#body 3-grams
gramlist=[]
for text in boiler_body:
    if text is not None:
        three_grams=ngrams(text.split(), 3)
        grams= [' '.join(gram) for gram in three_grams]
        gramlist.append(grams)
    else:
        gramlist.append(None)
                
grams=[] # cleanup for next trial
body_3grams = pd.Series(gramlist)
len(gramlist)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: generator 'ngrams' raised StopIteration
  


7395

In [105]:
df

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label,title_3grams,url_3grams,body_3grams
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,...,0,5424,170,8,0.152941,0.079130,0,"[ibm sees holographic, sees holographic calls,...","[a sign stands, sign stands outside, stands ou...","[a sign stands, sign stands outside, stands ou..."
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,...,0,4973,187,9,0.181818,0.125448,1,"[the fully electronic, fully electronic futuri...","[and that can, that can be, can be carried, be...","[and that can, that can be, can be carried, be..."
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,...,0,2240,258,11,0.166667,0.057613,1,"[fruits that fight, that fight the, fight the ...","[apples the most, the most popular, most popul...","[apples the most, the most popular, most popul..."
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,...,0,2737,120,5,0.041667,0.100858,1,"[10 foolproof tips, foolproof tips for, tips f...","[there was a, was a period, a period in, perio...","[there was a, was a period, a period in, perio..."
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,...,0,12032,162,10,0.098765,0.082569,0,"[the 50 coolest, 50 coolest jerseys, coolest j...","[jersey sales is, sales is a, is a curious, a ...","[jersey sales is, sales is a, is a curious, a ..."
5,http://www.conveniencemedical.com/genital-herp...,7018,"{""url"":""conveniencemedical genital herpes home...",?,?,119.000000,0.745455,0.581818,0.290909,0.018182,...,?,4368,55,3,0.054545,0.087356,0,[genital herpes treatment],"[genital herpes is, herpes is caused, is cause...","[genital herpes is, herpes is caused, is cause..."
6,http://gofashionlane.blogspot.tw/2012/06/ameri...,8685,"{""title"":""fashion lane American Wild Child "",""...",arts_entertainment,0.22111,0.773810,0.215054,0.053763,0.043011,0.043011,...,0,1287,93,3,0.548387,0.064327,1,"[fashion lane american, lane american wild, am...","[our favorite summer, favorite summer holiday,...","[our favorite summer, favorite summer holiday,..."
7,http://www.insidershealth.com/article/racing_f...,3402,"{""url"":""insidershealth article racing for reco...",?,?,1.883333,0.719697,0.265152,0.113636,0.015152,...,?,27656,132,4,0.068182,0.148551,0,"[racing for recovery, for recovery by, recover...","[racing for recovery, for recovery is, recover...","[racing for recovery, for recovery is, recover..."
8,http://www.valetmag.com/the-handbook/features/...,477,"{""title"":""Valet The Handbook 31 Days 31 days"",...",?,?,0.471503,0.190722,0.036082,0.000000,0.000000,...,0,2471,194,7,0.644330,0.125000,1,"[valet the handbook, the handbook 31, handbook...","[resolutions are for, are for suckers, for suc...","[resolutions are for, are for suckers, for suc..."
9,http://www.howsweeteats.com/2010/03/24/cookies...,6731,"{""url"":""howsweeteats 2010 03 24 cookies and cr...",?,?,2.410112,0.469325,0.101227,0.018405,0.003067,...,?,11459,326,4,0.236196,0.094412,1,"[cookies and cream, and cream brownies, cream ...","[more brownies it, brownies it seems, it seems...","[more brownies it, brownies it seems, it seems..."


In [113]:
se_titlegrams = pd.Series(title_3grams)
se_urlgrams = pd.Series(url_3grams)
se_bodygrams = pd.Series(body_3grams)
df['title_3grams'] = se_titlegrams.values
df['url_3grams'] = se_urlgrams.values
df['body_3grams'] = se_bodygrams.values

df

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label,title_3grams,url_3grams,body_3grams
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,...,0,5424,170,8,0.152941,0.079130,0,"[ibm sees holographic, sees holographic calls,...","[bloomberg news 2010, news 2010 12, 2010 12 23...","[a sign stands, sign stands outside, stands ou..."
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,...,0,4973,187,9,0.181818,0.125448,1,"[fully electronic futuristic, electronic futur...","[popsci technology article, technology article...","[and that can, that can be, can be carried, be..."
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,...,0,2240,258,11,0.166667,0.057613,1,"[fruits fight flu, fight flu fruits, flu fruit...","[menshealth health flu, health flu fighting, f...","[apples the most, the most popular, most popul..."
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,...,0,2737,120,5,0.041667,0.100858,1,"[10 foolproof tips, foolproof tips better, tip...","[dumblittleman 2007 12, 2007 12 10, 12 10 fool...","[there was a, was a period, a period in, perio..."
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,...,0,12032,162,10,0.098765,0.082569,0,"[50 coolest jerseys, coolest jerseys know, jer...","[bleacherreport articles 1205138, articles 120...","[jersey sales is, sales is a, is a curious, a ..."
5,http://www.conveniencemedical.com/genital-herp...,7018,"{""url"":""conveniencemedical genital herpes home...",?,?,119.000000,0.745455,0.581818,0.290909,0.018182,...,?,4368,55,3,0.054545,0.087356,0,[genital herpes treatment],"[conveniencemedical genital herpes, genital he...","[genital herpes is, herpes is caused, is cause..."
6,http://gofashionlane.blogspot.tw/2012/06/ameri...,8685,"{""title"":""fashion lane American Wild Child "",""...",arts_entertainment,0.22111,0.773810,0.215054,0.053763,0.043011,0.043011,...,0,1287,93,3,0.548387,0.064327,1,"[fashion lane american, lane american wild, am...","[gofashionlane blogspot tw, blogspot tw 2012, ...","[our favorite summer, favorite summer holiday,..."
7,http://www.insidershealth.com/article/racing_f...,3402,"{""url"":""insidershealth article racing for reco...",?,?,1.883333,0.719697,0.265152,0.113636,0.015152,...,?,27656,132,4,0.068182,0.148551,0,"[racing recovery dean, recovery dean johnson, ...","[insidershealth article racing, article racing...","[racing for recovery, for recovery is, recover..."
8,http://www.valetmag.com/the-handbook/features/...,477,"{""title"":""Valet The Handbook 31 Days 31 days"",...",?,?,0.471503,0.190722,0.036082,0.000000,0.000000,...,0,2471,194,7,0.644330,0.125000,1,"[valet handbook 31, handbook 31 days, 31 days ...","[valetmag handbook features, handbook features...","[resolutions are for, are for suckers, for suc..."
9,http://www.howsweeteats.com/2010/03/24/cookies...,6731,"{""url"":""howsweeteats 2010 03 24 cookies and cr...",?,?,2.410112,0.469325,0.101227,0.018405,0.003067,...,?,11459,326,4,0.236196,0.094412,1,"[cookies cream brownies, cream brownies sweet]","[howsweeteats 2010 03, 2010 03 24, 03 24 cooki...","[more brownies it, brownies it seems, it seems..."


In [130]:
df

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label,title_3grams,url_3grams,body_3grams
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,...,0,5424,170,8,0.152941,0.079130,0,"[ibm sees holographic, sees holographic calls,...","[bloomberg news 2010, news 2010 12, 2010 12 23...","[sign stands outside, stands outside internati..."
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,...,0,4973,187,9,0.181818,0.125448,1,"[fully electronic futuristic, electronic futur...","[popsci technology article, technology article...","[carried plane without, plane without hassle, ..."
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,...,0,2240,258,11,0.166667,0.057613,1,"[fruits fight flu, fight flu fruits, flu fruit...","[menshealth health flu, health flu fighting, f...","[apples popular source, popular source antioxi..."
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,...,0,2737,120,5,0.041667,0.100858,1,"[10 foolproof tips, foolproof tips better, tip...","[dumblittleman 2007 12, 2007 12 10, 12 10 fool...","[period life lot, life lot problems, lot probl..."
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,...,0,12032,162,10,0.098765,0.082569,0,"[50 coolest jerseys, coolest jerseys know, jer...","[bleacherreport articles 1205138, articles 120...","[jersey sales curious, sales curious business,..."
5,http://www.conveniencemedical.com/genital-herp...,7018,"{""url"":""conveniencemedical genital herpes home...",?,?,119.000000,0.745455,0.581818,0.290909,0.018182,...,?,4368,55,3,0.054545,0.087356,0,[genital herpes treatment],"[conveniencemedical genital herpes, genital he...","[genital herpes caused, herpes caused herpes, ..."
6,http://gofashionlane.blogspot.tw/2012/06/ameri...,8685,"{""title"":""fashion lane American Wild Child "",""...",arts_entertainment,0.22111,0.773810,0.215054,0.053763,0.043011,0.043011,...,0,1287,93,3,0.548387,0.064327,1,"[fashion lane american, lane american wild, am...","[gofashionlane blogspot tw, blogspot tw 2012, ...","[favorite summer holiday, summer holiday aroun..."
7,http://www.insidershealth.com/article/racing_f...,3402,"{""url"":""insidershealth article racing for reco...",?,?,1.883333,0.719697,0.265152,0.113636,0.015152,...,?,27656,132,4,0.068182,0.148551,0,"[racing recovery dean, recovery dean johnson, ...","[insidershealth article racing, article racing...","[racing recovery growing, recovery growing ide..."
8,http://www.valetmag.com/the-handbook/features/...,477,"{""title"":""Valet The Handbook 31 Days 31 days"",...",?,?,0.471503,0.190722,0.036082,0.000000,0.000000,...,0,2471,194,7,0.644330,0.125000,1,"[valet handbook 31, handbook 31 days, 31 days ...","[valetmag handbook features, handbook features...","[resolutions suckers instead, suckers instead ..."
9,http://www.howsweeteats.com/2010/03/24/cookies...,6731,"{""url"":""howsweeteats 2010 03 24 cookies and cr...",?,?,2.410112,0.469325,0.101227,0.018405,0.003067,...,?,11459,326,4,0.236196,0.094412,1,"[cookies cream brownies, cream brownies sweet]","[howsweeteats 2010 03, 2010 03 24, 03 24 cooki...","[brownies seems get, seems get one, get one fu..."


In [128]:
df

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label,title_3grams,url_3grams,body_3grams
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,...,0,5424,170,8,0.152941,0.079130,0,"[ibm sees holographic, sees holographic calls,...","[bloomberg news 2010, news 2010 12, 2010 12 23...","[sign stands outside, stands outside internati..."
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,...,0,4973,187,9,0.181818,0.125448,1,"[fully electronic futuristic, electronic futur...","[popsci technology article, technology article...","[carried plane without, plane without hassle, ..."
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,...,0,2240,258,11,0.166667,0.057613,1,"[fruits fight flu, fight flu fruits, flu fruit...","[menshealth health flu, health flu fighting, f...","[apples popular source, popular source antioxi..."
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,...,0,2737,120,5,0.041667,0.100858,1,"[10 foolproof tips, foolproof tips better, tip...","[dumblittleman 2007 12, 2007 12 10, 12 10 fool...","[period life lot, life lot problems, lot probl..."
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,...,0,12032,162,10,0.098765,0.082569,0,"[50 coolest jerseys, coolest jerseys know, jer...","[bleacherreport articles 1205138, articles 120...","[jersey sales curious, sales curious business,..."
5,http://www.conveniencemedical.com/genital-herp...,7018,"{""url"":""conveniencemedical genital herpes home...",?,?,119.000000,0.745455,0.581818,0.290909,0.018182,...,?,4368,55,3,0.054545,0.087356,0,[genital herpes treatment],"[conveniencemedical genital herpes, genital he...","[genital herpes caused, herpes caused herpes, ..."
6,http://gofashionlane.blogspot.tw/2012/06/ameri...,8685,"{""title"":""fashion lane American Wild Child "",""...",arts_entertainment,0.22111,0.773810,0.215054,0.053763,0.043011,0.043011,...,0,1287,93,3,0.548387,0.064327,1,"[fashion lane american, lane american wild, am...","[gofashionlane blogspot tw, blogspot tw 2012, ...","[favorite summer holiday, summer holiday aroun..."
7,http://www.insidershealth.com/article/racing_f...,3402,"{""url"":""insidershealth article racing for reco...",?,?,1.883333,0.719697,0.265152,0.113636,0.015152,...,?,27656,132,4,0.068182,0.148551,0,"[racing recovery dean, recovery dean johnson, ...","[insidershealth article racing, article racing...","[racing recovery growing, recovery growing ide..."
8,http://www.valetmag.com/the-handbook/features/...,477,"{""title"":""Valet The Handbook 31 Days 31 days"",...",?,?,0.471503,0.190722,0.036082,0.000000,0.000000,...,0,2471,194,7,0.644330,0.125000,1,"[valet handbook 31, handbook 31 days, 31 days ...","[valetmag handbook features, handbook features...","[resolutions suckers instead, suckers instead ..."
9,http://www.howsweeteats.com/2010/03/24/cookies...,6731,"{""url"":""howsweeteats 2010 03 24 cookies and cr...",?,?,2.410112,0.469325,0.101227,0.018405,0.003067,...,?,11459,326,4,0.236196,0.094412,1,"[cookies cream brownies, cream brownies sweet]","[howsweeteats 2010 03, 2010 03 24, 03 24 cooki...","[brownies seems get, seems get one, get one fu..."


In [129]:
df.to_csv("train-stumble_upon_boiler-3grams-filtered-nw.tsv", sep="\t")

In [84]:
len(boiler_title)

7395

In [62]:
new_df=pd.read_csv("train-stumble_upon_boiler-filtered.tsv", sep="\t")
new_df

,Unnamed: 0,Unnamed: 0.1,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,...,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label,boiler_body,boiler_title,boiler_url,title_sentiment,body_sentiment
0,0,0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,...,170,8,0.152941,0.079130,0,sign stands outside international business mac...,ibm sees holographic calls air breathing batte...,bloomberg news 2010 12 23 ibm predicts hologra...,0.096909,0.096909
1,1,1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,...,187,9,0.181818,0.125448,1,carried plane without hassle omega e gun start...,fully electronic futuristic starting gun elimi...,popsci technology article 2012 07 electronic f...,0.168040,0.168040
2,2,2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,...,258,11,0.166667,0.057613,1,apples popular source antioxidants diet one ap...,fruits fight flu fruits fight flu cold flu men...,menshealth health flu fighting fruits cm mmc F...,0.029092,0.029092
3,3,3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,...,120,5,0.041667,0.100858,1,period life lot problems sleep took long fall ...,10 foolproof tips better sleep,dumblittleman 2007 12 10 foolproof tips for be...,0.060979,0.060979
4,4,4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,...,162,10,0.098765,0.082569,0,jersey sales curious business whether buying s...,50 coolest jerseys know existed coolest jersey...,bleacherreport articles 1205138 the 50 coolest...,0.162464,0.162464
5,5,5,http://www.conveniencemedical.com/genital-herp...,7018,"{""url"":""conveniencemedical genital herpes home...",?,?,119.000000,0.745455,0.581818,...,55,3,0.054545,0.087356,0,genital herpes caused herpes simplex virus hsv...,genital herpes treatment,conveniencemedical genital herpes home php,0.061286,0.061286
6,6,6,http://gofashionlane.blogspot.tw/2012/06/ameri...,8685,"{""title"":""fashion lane American Wild Child "",""...",arts_entertainment,0.22111,0.773810,0.215054,0.053763,...,93,3,0.548387,0.064327,1,favorite summer holiday around corner boy heav...,fashion lane american wild child,gofashionlane blogspot tw 2012 06 american wil...,0.265909,0.265909
7,7,7,http://www.insidershealth.com/article/racing_f...,3402,"{""url"":""insidershealth article racing for reco...",?,?,1.883333,0.719697,0.265152,...,132,4,0.068182,0.148551,0,racing recovery growing idea drug addiction al...,racing recovery dean johnson racing recovery d...,insidershealth article racing for recovery 3471,0.239091,0.239091
8,8,8,http://www.valetmag.com/the-handbook/features/...,477,"{""title"":""Valet The Handbook 31 Days 31 days"",...",?,?,0.471503,0.190722,0.036082,...,194,7,0.644330,0.125000,1,resolutions suckers instead swearing sugar two...,valet handbook 31 days 31 days,valetmag the handbook features 31 days index p...,0.158475,0.158475
9,9,9,http://www.howsweeteats.com/2010/03/24/cookies...,6731,"{""url"":""howsweeteats 2010 03 24 cookies and cr...",?,?,2.410112,0.469325,0.101227,...,326,4,0.236196,0.094412,1,brownies seems get one full week without tryin...,cookies cream brownies sweet,howsweeteats 2010 03 24 cookies and cream brow...,0.213948,0.213948


## Try TF-IDF

In [262]:
import math
from textblob import TextBlob as tb

def tf(word, blob):
    if len(blob.words)==0:
        return 0
    else:
        return blob.words.count(word) / len(blob.words)

def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob.words)

def idf(word, bloblist):
    return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)

In [189]:
#berisihin boilerplate
ignorechars = [',', '.','-','--', '&', ';', ':', '?','#','>','<','=','[',']',')','(', '|','com','html']
filtered_text=[]
for text in boilerplate:
    if text is not None:
        tokens = get_tokens(text)
        filtered = [w for w in tokens if not w in ignorechars]
        filtered = [w for w in filtered if not w in stopwords.words('english')]
        filtered_text.append(filtered)
    else:
        filtered_text.append(None)
filtered_text[3]

['10',
 'foolproof',
 'tips',
 'better',
 'sleep',
 'period',
 'life',
 'lot',
 'problems',
 'sleep',
 'took',
 'long',
 'fall',
 'asleep',
 'easily',
 'awaken',
 'simply',
 'getting',
 'enough',
 'rest',
 'night',
 'want',
 'take',
 'medication',
 'led',
 'learn',
 'several',
 'tips',
 'tricks',
 'really',
 'helped',
 'overcome',
 'insomnia',
 'tips',
 'try',
 'follow',
 'regularly',
 'worry',
 'getting',
 'enough',
 'sleep',
 'try',
 'worry',
 'much',
 'sleep',
 'worrying',
 'start',
 'cycle',
 'negative',
 'thoughts',
 'contribute',
 'condition',
 'known',
 'learned',
 'insomnia',
 'learned',
 'insomnia',
 'occurs',
 'worry',
 'much',
 'whether',
 'able',
 'get',
 'adequate',
 'sleep',
 'bedtime',
 'rituals',
 'behavior',
 'actually',
 'trigger',
 'insomnia',
 'force',
 'sleep',
 'attempt',
 'trying',
 'actually',
 'awakes',
 'making',
 'difficult',
 'sleep',
 'go',
 'bed',
 'feeling',
 'really',
 'tired',
 'sleepy',
 'look',
 'alarm',
 'clock',
 'night',
 'looking',
 'clock',
 'pro

In [240]:
# join jadi satu text
boiler_text = []

for text in filtered_text:
    if text is not None:
        words = ' '.join(text)
        boiler_text.append(words)
    else:
        boiler_text.append(None)

print(boiler_text[1])

carried plane without hassle omega e gun starting pistol omega easy take granted insanely close olympic races much minutiae matter perfect example traditional starting gun seems easy pull trigger race starts boom people consider conventional gun goes sound travels ears closest runner fraction second sooner others enough matter latest starting pistol traded mechanical boom orchestrated electronic noise omega watch company tasked official timekeeper olympic games since 1932 2010 vancouver games debuted new starting gun far cry iconic revolvers associated early games clearly electronic still button pressed get show rolling far away get probably still clearly starting gun pull trigger olympians go pressed twice consecutively signals false start working speaker system eliminates kind advantage athletes big advantage close gun sound bullet traveling one meter every three milliseconds could contribute win powder pistols connected speaker system even runners could react sound real pistol firin

ValueError: Length of values does not match length of index

In [ ]:
urlid=df['urlid']
for i in enumerate(urlid):
    while count<20:
        a.insert(i, x)


In [252]:
len(scores)

47

In [253]:
df_notNull = df[pd.notnull(df['boiler_text'])]
df_notNull.head()
boiler_list=df_notNull['boiler_text']

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,lengthyLinkDomain,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label,boiler_text
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,...,1,24,0,5424,170,8,0.152941,0.079130,0,sign stands outside international business mac...
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,...,1,40,0,4973,187,9,0.181818,0.125448,1,carried plane without hassle omega e gun start...
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,...,1,55,0,2240,258,11,0.166667,0.057613,1,apples popular source antioxidants diet one ap...
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,...,0,24,0,2737,120,5,0.041667,0.100858,1,period life lot problems sleep took long fall ...
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,...,1,14,0,12032,162,10,0.098765,0.082569,0,jersey sales curious business whether buying s...


### compute tf_idf

In [254]:
boiler_list=df_notNull['boiler_text']

In [276]:
test_df.head()

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,lengthyLinkDomain,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label,boiler_text
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,...,1,24,0,5424,170,8,0.152941,0.079130,0,sign stands outside international business mac...
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,...,1,40,0,4973,187,9,0.181818,0.125448,1,carried plane without hassle omega e gun start...
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,...,1,55,0,2240,258,11,0.166667,0.057613,1,apples popular source antioxidants diet one ap...
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,...,0,24,0,2737,120,5,0.041667,0.100858,1,period life lot problems sleep took long fall ...
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,...,1,14,0,12032,162,10,0.098765,0.082569,0,jersey sales curious business whether buying s...


In [284]:
len(se_word)

7394

In [283]:
df_dummy=df_notNull
df_dummy['cooking']=se_word.values


TypeError: 'numpy.ndarray' object is not callable

In [278]:

score=[]
for i, blob in enumerate(bloblist):
    score.append(tfidf('cooking', blob, bloblist))
    print(i)
se_word = pd.Series(score)
len(se_word)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701


5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343


6783
6784
6785
6786
6787
6788
6789
6790
6791
6792
6793
6794
6795
6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980
6981
6982


7394

In [301]:
def get_tfidf(word, bloblist):
    scores=[]
    for i, blob in enumerate(bloblist):
        scores.append(tfidf(word, blob, bloblist))
    se_word = pd.Series(scores)
    return se_word.values

In [264]:
a = 'boiler_text'
df[a].head()

0    sign stands outside international business mac...
1    carried plane without hassle omega e gun start...
2    apples popular source antioxidants diet one ap...
3    period life lot problems sleep took long fall ...
4    jersey sales curious business whether buying s...
Name: boiler_text, dtype: object

In [292]:
import math
from textblob import TextBlob as tb

def tf(word, blob):
    if len(blob.words)==0:
        return 0
    else:
        return blob.words.count(word) / len(blob.words)

def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob.words)

def idf(word, bloblist):
    return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)

In [ ]:
# word dictionary

In [289]:
wol=['cooking', 'baking', 'top', 'food', 'look', 'day', '10', 'eat', 'try', 'serve', 'cool', 'little', 're',
          'heat', '12', 'butter', 'video', 'read', 'help', 'call', 'week', 'love', 'add', 'sugar', 'minutes',
          'start', 'run', 'love', 'eggs', 'flavor','cheese', 'own', '2011', '20', 'quick','brown', 'home',
          'chocolate', 'mix', 'people', 'tablespoons', 'blog', 'completely', 'water', 'including', 'ingredients', 
          'prepared', 'preheat', 'natural', 'friends', 'create', 'remove', 'inspired', 'recipes', 'ice', 'remove',
          'test', 'salt', 'line', 'write', 'news', 'cup', 'dish', 'food', 'fashion', 'article', 'image', 'milk',
          'light', 'store', 'slice','person','family','season','hand','create','pan', 'fun', 'hit', 'author', 
           'teaspoon', 'months', 'heavy', 'game', 'kitchen', 'recently', 'alternative', 'paper','slice','bowl',
          'cream', 'combine', 'vegetable', 'follow', 'roll', 'filling']

In [302]:
bloblist=[]
for text in boiler_list:
    bloblist.append(tb(text))

df_notNull['cooking']=get_tfidf('cooking', bloblist)

KeyboardInterrupt: 

In [296]:
# drop unimportant column to reduce file size
df_notNull = df.drop('boiler_text', 1)
df_notNull.to_csv('train_with_tfidf')

In [298]:
df_notNull['filling']

KeyError: 'filling'

In [ ]:
bloblist=[]
for text in boiler_text:
    if text is not None:
        bloblist.append(tb(text))
    else:
        pass
        
for i, blob in enumerate(bloblist):
    print("Top words in document {}".format(i + 1))
    scores = {word: tfidf(word, blob, bloblist) for word in blob.words}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    for word, score in sorted_words[:20]:
        print("\tWord: {}, TF-IDF: {}".format(word, round(score, 5)))


In [308]:
wordlist=['cooking', 'baking', 'top', 'food', 'look', 'day', '10', 'eat', 'try', 'serve', 'cool', 'little', 're',
          'heat', '12', 'butter', 'video', 'read', 'help', 'call', 'week', 'love', 'add', 'sugar', 'minutes',
          'start', 'run', 'love', 'eggs', 'flavor','cheese', 'own', '2011', '20', 'quick','brown', 'home',
          'chocolate', 'mix', 'people', 'tablespoons', 'blog', 'completely', 'water', 'including', 'ingredients', 
          'prepared', 'preheat', 'natural', 'friends', 'remove', 'inspired', 'recipes', 'ice',
          'test', 'salt', 'line', 'write', 'news', 'cup', 'dish', 'food', 'fashion', 'article', 'image', 'milk',
          'light', 'store', 'slice','person','family','season','hand','create','pan', 'fun', 'hit', 'author', 
           'teaspoon', 'months', 'heavy', 'game', 'kitchen', 'recently', 'alternative', 'paper','slice','bowl',
          'cream', 'combine', 'vegetable', 'follow', 'roll', 'filling', 'remove', 'available', 'test', 'offer', 'near', '50', 'package', 'feel', 'purpose',
          'step', 'bring', 'hot', 'smooth', '2012', '30','sweet', 'thanks', 
          'juice', 'pepper', 'technology', 'share', 'coat', 'increase', 
          'continue','care','list','chopped', 'features', 'delicious', 'form', 'product']

In [316]:
def clean_text(text):
	filtered_text=[]
	for t in text:
		if t is not None:
			tokens = get_tokens(t)
			filtered = [w for w in tokens if not w in stopwords.words('english') and w in wordlist]
			filtered_text.append(filtered)
		else:
			filtered_text.append(None)
	return (filtered_text)

In [321]:
def list_to_string(l):
	pool = []

	for text in l:
		if text is not None:
			words = ' '.join(text)
			pool.append(words)
		else:
			pool.append(None)
	return (pool)

In [8]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
#print('start cleaning text')
#f_boiler_text=clean_text(boiler_text)
#print('cleaning done')
boilerplate=[]
print('getting rid of Null values')
for text in boiler_text:
    if text is not None:
        boilerplate.append(text)
    else:
        pass
# Create text
print('create text')
text_data = np.array(boilerplate)
print(boilerplate[1])
print(text_data[1])


getting rid of Null values
create text
carried plane without hassle omega e gun starting pistol omega easy take granted insanely close olympic races much minutiae matter perfect example traditional starting gun seems easy pull trigger race starts boom people consider conventional gun goes sound travels ears closest runner fraction second sooner others enough matter latest starting pistol traded mechanical boom orchestrated electronic noise omega watch company tasked official timekeeper olympic games since 1932 2010 vancouver games debuted new starting gun far cry iconic revolvers associated early games clearly electronic still button pressed get show rolling far away get probably still clearly starting gun pull trigger olympians go pressed twice consecutively signals false start working speaker system eliminates kind advantage athletes big advantage close gun sound bullet traveling one meter every three milliseconds could contribute win powder pistols connected speaker system even runn

In [324]:
print('converting tokens to string')
boiler_str=list_to_string(boilerplate)
print('done')
print('creating np array')
text_data = np.array(boiler_str)
print(text_data[1])

converting tokens to string
done
creating np array
people start start look look start people home technology technology 2012


In [9]:
# Create the tf-idf feature matrix
print('create tf-idf feature matrix')
tfidf = TfidfVectorizer()
feature_matrix = tfidf.fit_transform(text_data)

# Show tf-idf feature matrix
feature_matrix

create tf-idf feature matrix


<7395x86143 sparse matrix of type '<class 'numpy.float64'>'
	with 1155519 stored elements in Compressed Sparse Row format>

In [10]:
# Show tf-idf feature matrix as dense matrix
tfidf_arr = feature_matrix.toarray()

In [19]:
# limit decimal places
np.round_(tfidf_arr,decimals=3)

array([[ 0.   ,  0.016,  0.   , ...,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   , ...,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   , ...,  0.   ,  0.   ,  0.   ],
       ..., 
       [ 0.   ,  0.   ,  0.   , ...,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   , ...,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   , ...,  0.   ,  0.   ,  0.   ]])

In [20]:
# create df baru buat tfidf, merge sama df lama
tfidf_arr.shape

(7395, 86143)

In [368]:
#duplicate df without null values
df_notNull = df[pd.notnull(df['boiler_text'])]
df_notNull = df.drop('boiler_text', 1)
df_notNull.head()

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,is_news,lengthyLinkDomain,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,...,1,1,24,0,5424,170,8,0.152941,0.079130,0
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,...,1,1,40,0,4973,187,9,0.181818,0.125448,1
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,...,1,1,55,0,2240,258,11,0.166667,0.057613,1
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,...,1,0,24,0,2737,120,5,0.041667,0.100858,1
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,...,1,1,14,0,12032,162,10,0.098765,0.082569,0


In [400]:
df.to_csv('train-with-boilertext.tsv', sep='\t')

In [2]:
import pandas as pd
from textblob import TextBlob
import json
df = pd.read_csv("train-with-boilertext.tsv", sep="\t")

In [7]:
boiler_text=df['boiler_text']

In [3]:
df.drop('boilerplate', 1)

,Unnamed: 0,url,urlid,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,lengthyLinkDomain,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label,boiler_text
0,0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,...,1,24,0,5424,170,8,0.152941,0.079130,0,sign stands outside international business mac...
1,1,http://www.popsci.com/technology/article/2012-...,8471,recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,...,1,40,0,4973,187,9,0.181818,0.125448,1,carried plane without hassle omega e gun start...
2,2,http://www.menshealth.com/health/flu-fighting-...,1164,health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,...,1,55,0,2240,258,11,0.166667,0.057613,1,apples popular source antioxidants diet one ap...
3,3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,...,0,24,0,2737,120,5,0.041667,0.100858,1,period life lot problems sleep took long fall ...
4,4,http://bleacherreport.com/articles/1205138-the...,9006,sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,...,1,14,0,12032,162,10,0.098765,0.082569,0,jersey sales curious business whether buying s...
5,5,http://www.conveniencemedical.com/genital-herp...,7018,?,?,119.000000,0.745455,0.581818,0.290909,0.018182,...,0,12,?,4368,55,3,0.054545,0.087356,0,genital herpes caused herpes simplex virus hsv...
6,6,http://gofashionlane.blogspot.tw/2012/06/ameri...,8685,arts_entertainment,0.22111,0.773810,0.215054,0.053763,0.043011,0.043011,...,0,21,0,1287,93,3,0.548387,0.064327,1,favorite summer holiday around corner boy heav...
7,7,http://www.insidershealth.com/article/racing_f...,3402,?,?,1.883333,0.719697,0.265152,0.113636,0.015152,...,0,5,?,27656,132,4,0.068182,0.148551,0,racing recovery growing idea drug addiction al...
8,8,http://www.valetmag.com/the-handbook/features/...,477,?,?,0.471503,0.190722,0.036082,0.000000,0.000000,...,0,17,0,2471,194,7,0.644330,0.125000,1,resolutions suckers instead swearing sugar two...
9,9,http://www.howsweeteats.com/2010/03/24/cookies...,6731,?,?,2.410112,0.469325,0.101227,0.018405,0.003067,...,1,14,?,11459,326,4,0.236196,0.094412,1,brownies seems get one full week without tryin...


In [21]:


# create df from numpy array containint tfidf, append urlid as column
df_tf=pd.DataFrame(data=tfidf_arr)
df_tf['urlid']=df['urlid']
df_tf.head()

,0,1,2,3,4,5,6,7,8,9,...,86134,86135,86136,86137,86138,86139,86140,86141,86142,urlid
0,0.0,0.015801,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4042
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8471
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1164
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6684
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9006


In [22]:
merged_df=pd.merge(df, df_tf, how='inner', on='urlid',
         left_index=False)
merged_df=merged_df.drop('boilerplate', 1)
merged_df.head()

,Unnamed: 0,url,urlid,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,86133,86134,86135,86136,86137,86138,86139,86140,86141,86142
0,0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,http://www.popsci.com/technology/article/2012-...,8471,recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,http://www.menshealth.com/health/flu-fighting-...,1164,health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,http://bleacherreport.com/articles/1205138-the...,9006,sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
merged_df.to_csv('train-with-full-tfidf.tsv', sep='\t')

In [350]:
len(merged_df)

7394

In [388]:
merged_df[86133].values

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [382]:
boilerplate=[]
for text in boiler_text:
    if text is not None:
        boilerplate.append(text)
    else:
        pass
# Create text
text_data = np.array(boilerplate)

# Create the tf-idf feature matrix
tfidf = TfidfVectorizer()
feature_matrix = tfidf.fit_transform(text_data)

# Show tf-idf feature matrix
feature_matrix
# Show tf-idf feature matrix as dense matrix
tfidf_arr = feature_matrix.toarray()
tfidf_arr.shape

(7394, 86143)

In [385]:
np.unique(tfidf_arr

array([  0.00000000e+00,   4.87906283e-04,   5.02106745e-04, ...,
         9.70257839e-01,   9.96004318e-01,   1.00000000e+00])

In [398]:
for i in range(86143):
    if (merged_df[i].sum() < 0.005):
        merged_df = merged_df.drop(i, 1)
        print('%d dropped' %i)
    else:
        pass
merged_df.head()

45 dropped
46 dropped
47 dropped
102 dropped
136 dropped
234 dropped
276 dropped
280 dropped
281 dropped
307 dropped
310 dropped
325 dropped
328 dropped
330 dropped
333 dropped
334 dropped
339 dropped
340 dropped
341 dropped
350 dropped
351 dropped
355 dropped
359 dropped
377 dropped
378 dropped
385 dropped
398 dropped
400 dropped
401 dropped
406 dropped
420 dropped
484 dropped
485 dropped
487 dropped
489 dropped
490 dropped
502 dropped
503 dropped
506 dropped
515 dropped
519 dropped
520 dropped
526 dropped
528 dropped
529 dropped
536 dropped
540 dropped
542 dropped
544 dropped
545 dropped
549 dropped
551 dropped
554 dropped
555 dropped
566 dropped
568 dropped
582 dropped
583 dropped
584 dropped
590 dropped
591 dropped
592 dropped
593 dropped
595 dropped
597 dropped
601 dropped
607 dropped
610 dropped
643 dropped
644 dropped
645 dropped
647 dropped
651 dropped
652 dropped
653 dropped
654 dropped
670 dropped
672 dropped
673 dropped
674 dropped
680 dropped
685 dropped
686 dropped
688 dro

KeyboardInterrupt: 

In [24]:
df_test=pd.read_csv('Stumble_Upon/test-with-boilertext.tsv', sep='\t')
df_test.head()

,Unnamed: 0,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,...,is_news,lengthyLinkDomain,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,boiler_text
0,0,http://www.lynnskitchenadventures.com/2009/04/...,5865,"{""title"":""Homemade Enchilada Sauce Lynn s Kitc...",recreation,0.443906,2.558140,0.389706,0.257353,0.044118,...,1,1,15,0,5643,136,3,0.242647,0.080597,i usually buy my enchilada sauce yes i knew i ...
1,1,http://lolpics.se/18552-stun-grenade-ar,782,"{""title"":""lolpics Stun grenade ar "",""body"":"" f...",culture_politics,0.135844,3.771429,0.461538,0.205128,0.051282,...,?,1,62,0,382,39,2,0.128205,0.176471,funny pictures at lolpics.se . the best funny ...
2,2,http://www.xcelerationfitness.com/treadmills.html,6962,"{""title"":""Treadmills "",""body"":"" treadmills, st...",?,?,2.269565,0.495726,0.384615,0.170940,...,?,1,42,0,2420,117,1,0.581197,0.125000,"treadmills , stair , climbers , treadmills tre..."
3,3,http://www.bloomberg.com/news/2012-02-06/syria...,7640,"{""title"":""Father s Tactics Used by Assad to Cr...",culture_politics,0.90259,2.523490,0.705502,0.346278,0.122977,...,1,1,41,0,5559,309,10,0.038835,0.063126,enlarge image syrian president bashar al assad...
4,4,http://www.wired.com/gadgetlab/2011/12/stem-tu...,3589,"{""title"":""Stem Turns Lemons and Limes Into Jui...",science_technology,0.486363,1.848000,0.470968,0.161290,0.032258,...,1,0,34,0,2209,155,10,0.096774,0.065341,quirky s stem turns any citrus fruit into an a...
